In [59]:
import pandas as pd
import panel as pn
import hvplot.pandas

df = pd.read_csv('C:\\tramites-migratorios.csv')
order = ['Resuelto Positivamente','Sin Resolucion','Resuelto Negativamente','Desechado','Desistimiento por parte del solicitante','Tramite cancelado','Autorizado y enviado a SRE']

df['Tipo de Resolucion'] = pd.Categorical(df['Tipo de Resolucion'], categories=order, ordered=True)

conteo_trimestre_tipo = df.groupby(['Trimestre', 'Pais de Origen', 'Tipo de Resolucion']).size().reset_index(name='conteo')

trimestre_select = pn.widgets.Select(name='Trimestre', options=['Todos'] + list(df['Trimestre'].unique()))
pais_select = pn.widgets.Select(name='Pais de Origen', options=['Todos'] + list(df['Pais de Origen'].unique()))

@pn.depends(trimestre_select, pais_select)
def update_plot(trimestre, pais):
    if trimestre == 'Todos' and pais == 'Todos':
        filtered_df = conteo_trimestre_tipo
    elif trimestre == 'Todos':
        filtered_df = conteo_trimestre_tipo[conteo_trimestre_tipo['Pais de Origen'] == pais]
    elif pais == 'Todos':
        filtered_df = conteo_trimestre_tipo[conteo_trimestre_tipo['Trimestre'] == trimestre]
    else:
        filtered_df = conteo_trimestre_tipo[(conteo_trimestre_tipo['Trimestre'] == trimestre) & 
                                            (conteo_trimestre_tipo['Pais de Origen'] == pais)]
    
    if not filtered_df.empty:
        plot = filtered_df.hvplot.bar(x='Tipo de Resolucion', y='conteo', 
                                      title=f'Resoluciones por Tipo para {trimestre} - {pais}',
                                      xlabel='Tipo de Resolución', ylabel='Cantidad', 
                                      color='pink').opts(xrotation=45, width=800, height=400)
        return pn.pane.HoloViews(plot)
    else:
        return pn.pane.HTML('<div style="font-size:2em; text-align:center">No hay datos disponibles</div>')

dashboard = pn.Column(
    'Dashboard de Resoluciones por Tipo',
    pn.Row(trimestre_select, pais_select, align='center'),
    update_plot,
    align='center'
)

dashboard.show()


Launching server at http://localhost:55753
